In [46]:
import pandas as pd
import requests
from itertools import combinations

In [47]:
api_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    print("Succesfully fetched data!")
else:
    print(f"Error while fetching data. Status code: {response.status_code}")

Succesfully fetched data!


In [48]:
print(data.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [49]:
players = pd.DataFrame(data['elements'])
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,137,82,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,...,32,15,303,38,459,49,214,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,...,66,3,76,17,31,4,13,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,...,13,5,88,18,37,14,37,10,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,638,63,425,51,514,58,558,68,0.00,0.00


In [50]:
print("Players DataFrame Columns:")
print(players.columns.tolist())


Players DataFrame Columns:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_

In [51]:
position_map = {
    1: 'GK',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

players_selected = players[
    ['first_name', 'second_name', 'team_code', 'element_type', 'total_points', 'points_per_game', 'now_cost']
].copy()

players_selected['now_cost'] = players_selected['now_cost'] / 10

players_selected['element_type'] = players_selected['element_type'].map(position_map)

players_selected[['total_points', 'points_per_game', 'now_cost']] = players_selected[
    ['total_points', 'points_per_game', 'now_cost']
].apply(pd.to_numeric, errors='coerce')

players_selected = players_selected.loc[players_selected['points_per_game'] > 0.0]

players_selected['full_name'] = players_selected['first_name'] + ' ' + players_selected['second_name']

print(players_selected)


         first_name             second_name  team_code element_type  \
1           Gabriel       Fernando de Jesus          3      Forward   
2           Gabriel    dos Santos Magalhães          3     Defender   
3               Kai                 Havertz          3      Forward   
5           Jurriën                  Timber          3     Defender   
6        Jorge Luiz            Frello Filho          3   Midfielder   
..              ...                     ...        ...          ...   
686          Jørgen           Strand Larsen         39      Forward   
687    Toti António                   Gomes         39     Defender   
689           André  Trindade da Costa Neto         39   Midfielder   
690  Carlos Roberto            Forbs Borges         39   Midfielder   
691           Alfie                    Pond         39     Defender   

     total_points  points_per_game  now_cost                     full_name  
1               9              0.8       6.8     Gabriel Fernando de J

In [52]:
max_indeces = players_selected[['total_points', 'points_per_game', 'now_cost']].idxmax()
max_rows = players_selected.loc[max_indeces]
print(max_rows)


    first_name second_name  team_code element_type  total_points  \
401    Mohamed       Salah         14   Midfielder           151   
401    Mohamed       Salah         14   Midfielder           151   
428     Erling     Haaland         43      Forward            98   

     points_per_game  now_cost       full_name  
401             10.8      13.3   Mohamed Salah  
401             10.8      13.3   Mohamed Salah  
428              6.5      14.9  Erling Haaland  


In [53]:
forwards = players_selected[players_selected['element_type'] == 'Forward']

forwards.head()

,first_name,second_name,team_code,element_type,total_points,points_per_game,now_cost,full_name
1,Gabriel,Fernando de Jesus,3,Forward,9,0.8,6.8,Gabriel Fernando de Jesus
3,Kai,Havertz,3,Forward,59,4.2,7.9,Kai Havertz
43,Jhon,Durán,7,Forward,41,2.7,5.8,Jhon Durán
63,Ollie,Watkins,7,Forward,77,5.1,9.0,Ollie Watkins
77,Enes,Ünal,91,Forward,20,1.7,5.4,Enes Ünal


In [54]:
print(f"Total GK's: {len(forwards)}")

Total GK's: 51


In [55]:
min_indeces = forwards[['total_points', 'points_per_game', 'now_cost']].idxmin()
min_rows = forwards.loc[min_indeces]
print(min_rows)


    first_name second_name  team_code element_type  total_points  \
209       Marc    Guiu Paz          8      Forward             1   
318        Ali   Al-Hamadi         40      Forward             4   
574       Ross     Stewart         20      Forward             3   

     points_per_game  now_cost      full_name  
209              1.0       4.7  Marc Guiu Paz  
318              0.7       4.8  Ali Al-Hamadi  
574              0.8       4.4   Ross Stewart  


In [58]:
# Generate all combinations of 3 forwards
forward_combinations = list(combinations(forwards.itertuples(), 3))

print(len(forward_combinations))

optimal_forwards = None
best_total_points = 0
best_comb_cost = 0
max_cost = 30

for player1, player2, player3 in forward_combinations:
    total_cost = player1.now_cost + player2.now_cost + player3.now_cost
    if total_cost > max_cost:
        continue

    total_points = player1.total_points + player2.total_points + player3.total_points

    if total_points > best_total_points:
        best_total_points = total_points
        best_comb_cost = total_cost
        optimal_forwards = (player1, player2, player3)


# Unpack and print the results
print(f"Total points: {best_total_points}")
print(f"Total cost: {best_comb_cost} \n")
print(f"Cost/point: {best_comb_cost / best_total_points} \n")

for player in optimal_forwards:
    print(f"Player: {player.first_name} {player.second_name} - {player.element_type} - Team Code: {player.team_code} - Total points: {player.total_points} - Cost: £{player.now_cost}m")


20825
Total points: 266
Total cost: 29.6 

Cost/point: 0.11127819548872181 

Player: Nicolas Jackson - Forward - Team Code: 8 - Total points: 80 - Cost: £8.2m
Player: Erling Haaland - Forward - Team Code: 43 - Total points: 98 - Cost: £14.9m
Player: Chris Wood - Forward - Team Code: 17 - Total points: 88 - Cost: £6.5m
